# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from custom_transformer import StartingVerbExtractor

nltk.download(['words', 'punkt', 'stopwords',
               'averaged_perceptron_tagger',
               'maxent_ne_chunker', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayemyatwinshwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_d

True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('message_category', engine).sample(3000)

In [3]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3872,4437,I say thank you because I see that you underst...,mwen di mesi paskem we nou konprann sam di nou...,direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
19212,22433,And the Taliban refusal to address this seriou...,None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# number of distinct observations
df.nunique()

id                        2999
message                   2999
original                  1159
genre                        3
related                      2
request                      2
offer                        2
aid_related                  2
medical_help                 2
medical_products             2
search_and_rescue            2
security                     2
military                     2
child_alone                  1
water                        2
food                         2
shelter                      2
clothing                     2
money                        2
missing_people               2
refugees                     2
death                        2
other_aid                    2
infrastructure_related       2
transport                    2
buildings                    2
electricity                  2
tools                        2
hospitals                    2
shops                        2
aid_centers                  2
other_infrastructure         2
weather_

In [5]:
# number of missing values
df.isnull().sum()

id                           0
message                      0
original                  1829
genre                        0
related                      0
request                      0
offer                        0
aid_related                  0
medical_help                 0
medical_products             0
search_and_rescue            0
security                     0
military                     0
child_alone                  0
water                        0
food                         0
shelter                      0
clothing                     0
money                        0
missing_people               0
refugees                     0
death                        0
other_aid                    0
infrastructure_related       0
transport                    0
buildings                    0
electricity                  0
tools                        0
hospitals                    0
shops                        0
aid_centers                  0
other_infrastructure         0
weather_

In [6]:
# drop id, original
df.drop(['id', 'original'], axis=1, inplace=True)

In [7]:
df.head(2)

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3872,I say thank you because I see that you underst...,direct,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
19212,And the Taliban refusal to address this seriou...,news,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = df['message'].values
Y = df.loc[:,'related':'direct_report'].values

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # replace urls with placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # convert to lowercase and remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    # remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    # reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    # lemmatize verbs by specifying pos
    lemmed_tokens = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    # stem tokens
    cleaned_tokens = [PorterStemmer().stem(w) for w in lemmed_tokens]

    return cleaned_tokens

In [10]:
# Count the number of tokens

class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def text_len_count(self, text):

        text_length = len(tokenize(text))

        return text_length

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_text_len = pd.Series(X).apply(self.text_len_count)
        return pd.DataFrame(X_text_len)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, max_features=None, max_df=1.0)),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
])

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1bc2bb00>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                                      random_state=None),
                       

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42)

In [14]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1bc2bb00>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [15]:
Y_pred = pipeline.predict(X_test)

In [16]:
(Y_pred == Y_test).mean()

0.9375925925925926

In [17]:
def display_results(y_test, y_pred, y_col):
    
    clf_report = classification_report(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    accuracy = (y_pred == y_test).mean() 
    print('\n')   
    print(y_col, ":")
    print('\n')  
    print(clf_report)
    print(confusion_mat)
    print('\n')
    print('Accuracy =', accuracy)
    print('-'*65)

In [18]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:,i], Y_pred[:,i], df.loc[:,'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.53      0.27      0.35       218
           1       0.80      0.92      0.86       682

    accuracy                           0.76       900
   macro avg       0.66      0.59      0.60       900
weighted avg       0.73      0.76      0.73       900

[[ 58 160]
 [ 52 630]]


Accuracy = 0.7644444444444445
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.91      0.95      0.93       751
           1       0.68      0.53      0.59       149

    accuracy                           0.88       900
   macro avg       0.79      0.74      0.76       900
weighted avg       0.87      0.88      0.87       900

[[713  38]
 [ 70  79]]


Accuracy = 0.88
-----------------------------------------------------------------


offer :


              precision    recall  f1-score   support

           0       1.00



cold :


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       891
           1       0.44      0.44      0.44         9

    accuracy                           0.99       900
   macro avg       0.72      0.72      0.72       900
weighted avg       0.99      0.99      0.99       900

[[886   5]
 [  5   4]]


Accuracy = 0.9888888888888889
-----------------------------------------------------------------


other_weather :


              precision    recall  f1-score   support

           0       0.94      0.97      0.96       845
           1       0.21      0.13      0.16        55

    accuracy                           0.92       900
   macro avg       0.58      0.55      0.56       900
weighted avg       0.90      0.92      0.91       900

[[819  26]
 [ 48   7]]


Accuracy = 0.9177777777777778
-----------------------------------------------------------------


direct_report :


              precision    recall  f1-score   support

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
%timeit

parameters = {

    # 'vect__max_df': [0.75, 1.0],
    # 'vect__max_features': [None, 5000],
    # 'vect__ngram_range': [(1, 1), (1, 2)],
    # 'tfidf__smooth_idf': [True, False],
    # 'tfidf__sublinear_tf': [True, False],
    # 'tfidf__use_idf': [True, False],
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [20, 50]

}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1, verbose=2)

cv.fit(X_train, Y_train)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  32 | elapsed:  1.8min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
Y_pred_grid = cv.predict(X_test)

In [21]:
(Y_pred_grid == Y_test).mean()

0.9414814814814815

In [22]:
cv.get_params()

{'cv': 2,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a1bc2bb00>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                      base_estimator=None,
                                                      learning_rate=1.0,
                                                      n_estimators=50,
                                            

In [23]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:,i], Y_pred_grid[:,i], df.loc[:,'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.62      0.06      0.11       218
           1       0.77      0.99      0.86       682

    accuracy                           0.76       900
   macro avg       0.69      0.52      0.49       900
weighted avg       0.73      0.76      0.68       900

[[ 13 205]
 [  8 674]]


Accuracy = 0.7633333333333333
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.89      0.98      0.93       751
           1       0.79      0.37      0.50       149

    accuracy                           0.88       900
   macro avg       0.84      0.67      0.72       900
weighted avg       0.87      0.88      0.86       900

[[736  15]
 [ 94  55]]


Accuracy = 0.8788888888888889
-----------------------------------------------------------------


offer :


              precision    recall  f1-score   support

         

/Users/ayemyatwinshwe/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))




earthquake :


              precision    recall  f1-score   support

           0       0.97      0.99      0.98       816
           1       0.85      0.75      0.80        84

    accuracy                           0.96       900
   macro avg       0.91      0.87      0.89       900
weighted avg       0.96      0.96      0.96       900

[[805  11]
 [ 21  63]]


Accuracy = 0.9644444444444444
-----------------------------------------------------------------


cold :


              precision    recall  f1-score   support

           0       0.99      1.00      0.99       891
           1       0.00      0.00      0.00         9

    accuracy                           0.99       900
   macro avg       0.49      0.50      0.50       900
weighted avg       0.98      0.99      0.98       900

[[890   1]
 [  9   0]]


Accuracy = 0.9888888888888889
-----------------------------------------------------------------


other_weather :


              precision    recall  f1-score   support

 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
pipeline_2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize,
                                     max_features=None,
                                     max_df=1.0)),
            ('tfidf', TfidfTransformer(use_idf=True))
        ])),

        ('text_len', TextLengthExtractor())
    ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [25]:
pipeline_2.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

In [26]:
Y_pred_2 = pipeline_2.predict(X_test)

In [27]:
(Y_pred_2 == Y_test).mean()

0.9369444444444445

In [28]:
for i in range(Y_test.shape[1]):
    display_results(Y_test[:, i], Y_pred_2[:, i],
                    df.loc[:, 'related':'direct_report'].columns[i])



related :


              precision    recall  f1-score   support

           0       0.57      0.25      0.35       218
           1       0.80      0.94      0.86       682

    accuracy                           0.77       900
   macro avg       0.68      0.59      0.60       900
weighted avg       0.74      0.77      0.74       900

[[ 54 164]
 [ 41 641]]


Accuracy = 0.7722222222222223
-----------------------------------------------------------------


request :


              precision    recall  f1-score   support

           0       0.91      0.96      0.93       751
           1       0.70      0.52      0.59       149

    accuracy                           0.88       900
   macro avg       0.80      0.74      0.76       900
weighted avg       0.87      0.88      0.88       900

[[718  33]
 [ 72  77]]


Accuracy = 0.8833333333333333
-----------------------------------------------------------------


offer :


              precision    recall  f1-score   support

         



fire :


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       890
           1       0.00      0.00      0.00        10

    accuracy                           0.98       900
   macro avg       0.49      0.50      0.50       900
weighted avg       0.98      0.98      0.98       900

[[883   7]
 [ 10   0]]


Accuracy = 0.9811111111111112
-----------------------------------------------------------------


earthquake :


              precision    recall  f1-score   support

           0       0.97      0.98      0.98       816
           1       0.83      0.75      0.79        84

    accuracy                           0.96       900
   macro avg       0.90      0.87      0.88       900
weighted avg       0.96      0.96      0.96       900

[[803  13]
 [ 21  63]]


Accuracy = 0.9622222222222222
-----------------------------------------------------------------


cold :


              precision    recall  f1-score   support

          

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.